# Cryptocurrency Fundamental Analysis Tool

In [ ]:
import json
import requests
import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
from pathlib import Path


In [140]:

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2018-01-01").isoformat()
end_date = pd.Timestamp.today().isoformat()

# Number of records to retrieve
page_size = 6000

# Define the list of crypto assets to analyze - Unable to loop through CSV file
crypto_asset = ['btc', 'eth', 'doge', 'aave']

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in crypto_asset:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     metric: value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Set Date as the index
    data_df.index = data_df['Date']
    
    # Change the value type to 'float' to allow graphing
    data_df[metric] = data_df[metric].astype(float)
    
    # Define columns
    data_df = data_df.drop(columns='Date')
    data_df.columns = ['Token', metric]
    
    # Format the DataFrame to allow xgraphing
    #updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    #return updated_data_df
    
    return data_df
    

In [141]:
# Parse metrics
active_addresses = get_data('AdrActCnt')
transaction_count = get_data('TxCnt')
supply = get_data('SplyCur')
mc = get_data('CapMrktCurUSD')
mc_real = get_data('CapRealUSD')
velocity = get_data('VelCur1yr')

In [143]:
active_addresses[active_addresses['Token'] == crypto_asset[-1]]

,Token,AdrActCnt
Date,,
2020-09-24,aave,0.0
2020-09-25,aave,0.0
2020-09-26,aave,0.0
2020-09-27,aave,0.0
2020-09-28,aave,0.0
...,...,...
2021-09-30,aave,830.0
2021-10-01,aave,853.0
2021-10-02,aave,1022.0


In [137]:
df = pd.concat([active_addresses, transaction_count, supply, mc, mc_real, velocity], join='outer', axis=1)
df = df.loc[:, ~df.columns.duplicated()]
df

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [126]:
# UNABLE TO ITERATE THROUGH THE TOKEN LIST CSV FILE - ERROR WITH COINMETRICS?

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2018-01-01").isoformat()
end_date = pd.Timestamp.today().isoformat()

# Number of records to retrieve
page_size = 6000

# Define the list of crypto assets to analyze
crypto_asset = pd.read_csv(Path('./token_list.csv'), header=None)

tokens = token_list.values.tolist()

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in tokens:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     metric: value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Set Date as the index
    data_df.index = data_df['Date']
    
    # Change the value type to 'float' to allow graphing
    data_df[metric] = data_df[metric].astype(float)
    
    # Define columns
    data_df = data_df.drop(columns='Date')
    data_df.columns = ['Token', metric]
    
    # Format the DataFrame to allow xgraphing
    #updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    #return updated_data_df
    
    return data_df
    

In [127]:
get_data('AdrActCnt')

KeyError: 'data'